In [273]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [274]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [275]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [276]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/Users/saikirananugam/Desktop/nndl/NNDL/datasets.csv/Actor_*//*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [257]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.2)

/var/folders/cf/78m2pngj3kqgdk88fltk67r80000gn/T/ipykernel_2021/852816848.py:16: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
/var/folders/cf/78m2pngj3kqgdk88fltk67r80000gn/T/ipykernel_2021/852816848.py:16: FutureWarning: Pass y=[ 3.0517578e-05  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
 -3.0517578e-05 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
/var/folders/cf/78m2pngj3kqgdk88fltk67r80000gn/T/ipykernel_2021/852816848.py:16: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -3.0517578e-05
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.mels

In [258]:
x_train

array([[-5.39380310e+02,  3.26435585e+01, -1.24525795e+01, ...,
         1.57894555e-03,  5.58187428e-04,  2.49191973e-04],
       [-5.52237061e+02,  3.83860931e+01, -3.20024896e+00, ...,
         3.32614494e-04,  1.91174040e-04,  1.69044943e-04],
       [-6.66638855e+02,  5.00913048e+01, -9.36156178e+00, ...,
         2.66759434e-05,  1.44194864e-05,  8.54850714e-06],
       ...,
       [-6.51865662e+02,  7.38328705e+01, -2.77009106e+00, ...,
         6.39894552e-06,  5.42769430e-06,  2.27292230e-06],
       [-6.47903625e+02,  4.48624687e+01, -2.82318783e+00, ...,
         4.23921701e-05,  4.01655961e-05,  1.77828169e-05],
       [-6.93024231e+02,  5.46327591e+01,  1.00681839e+01, ...,
         3.62368592e-05,  1.73679909e-05,  6.85253144e-06]])

In [259]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(614, 154)


In [260]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [261]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08,hidden_layer_sizes=(300,300), learning_rate='adaptive', max_iter=500)

In [262]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300, 300),
              learning_rate='adaptive', max_iter=500)

In [263]:
#Predict for the test set
y_pred=model.predict(x_test)

In [264]:
y_pred

array(['calm', 'fearful', 'disgust', 'calm', 'disgust', 'fearful', 'calm',
       'disgust', 'disgust', 'calm', 'disgust', 'disgust', 'disgust',
       'disgust', 'happy', 'disgust', 'disgust', 'happy', 'disgust',
       'disgust', 'disgust', 'disgust', 'fearful', 'disgust', 'fearful',
       'disgust', 'fearful', 'calm', 'disgust', 'disgust', 'disgust',
       'disgust', 'disgust', 'disgust', 'calm', 'calm', 'disgust',
       'happy', 'disgust', 'happy', 'disgust', 'disgust', 'disgust',
       'happy', 'disgust', 'disgust', 'calm', 'disgust', 'calm',
       'disgust', 'happy', 'disgust', 'disgust', 'disgust', 'happy',
       'disgust', 'disgust', 'disgust', 'calm', 'disgust', 'happy',
       'happy', 'calm', 'disgust', 'fearful', 'disgust', 'disgust',
       'calm', 'disgust', 'fearful', 'disgust', 'calm', 'fearful',
       'disgust', 'calm', 'disgust', 'disgust', 'calm', 'disgust', 'calm',
       'disgust', 'fearful', 'fearful', 'disgust', 'calm', 'disgust',
       'calm', 'calm', 'c

In [265]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 54.55%


In [266]:
from sklearn.metrics import accuracy_score, f1_score

In [267]:
f1_score(y_test, y_pred,average=None)

array([0.61971831, 0.55882353, 0.36842105, 0.53968254])

In [268]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,calm,calm
1,fearful,fearful
2,disgust,disgust
3,disgust,calm
4,happy,disgust
5,fearful,fearful
6,calm,calm
7,happy,disgust
8,disgust,disgust
9,calm,calm


In [269]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [270]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/Users/saikirananugam/Desktop/team13/datasets.csv/Actor_10/03-01-01-01-01-01-10.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction[0]

/var/folders/cf/78m2pngj3kqgdk88fltk67r80000gn/T/ipykernel_2021/852816848.py:16: FutureWarning: Pass y=[6.1035156e-05 3.0517578e-05 6.1035156e-05 ... 0.0000000e+00 3.0517578e-05
 3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


'disgust'

In [271]:
prediction[0]

'disgust'

In [272]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.pkl', 'wb') as f:
    pickle.dump(model,f)